In [23]:
import pandas as pd

def clean_file(df):
    df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)
    for i in range(len(df)):
        start = df.at[i, 'Published Start']
        if start[-1] == 'a' or start[0:2] == '12':
            if len(start.split(':')[0]) == 1:
                df.at[i, 'Published Start'] = '0%s' %(start[:-1])
            else:
                df.at[i, 'Published Start'] = start[:-1]
        elif start == 'noon':
            df.at[i, 'Published Start'] = '12:00'
        else:
            hour = int(start.split(':')[0])
            hour += 12
            rest = start.split(':')[1]
            df.at[i, 'Published Start'] = '%d:%s' %(hour,rest[0:2])


        end = df.at[i, 'Published End']
        if end[-1] == 'a' or end[0:2] == '12':
            if len(end.split(':')[0]) == 1:
                df.at[i, 'Published End'] = '0%s' %(end[:-1])
            else:
                df.at[i, 'Published End'] = end[:-1]
        elif end == 'noon':
            df.at[i, 'Published End'] = '12:00'
        else:
            hour = int(end.split(':')[0])
            hour += 12
            rest = end.split(':')[1]
            df.at[i, 'Published End'] = '%d:%s' %(hour,rest[0:2])

        df.at[i, 'Building'] = df.at[i, 'Location'].split()[0]
        df.at[i, 'Room'] = df.at[i, 'Location'].split()[1]

def combine_time(df):
    for i in range(len(df)):
        df.at[i, "Start Time"] = df.at[0, "Date"] + "T" + df.at[0, "Published Start"]
        df.at[i, "End Time"] = df.at[0, "Date"] + "T" + df.at[0, "Published End"]

def convert_iso(df):
    df["Start Time"] = pd.to_datetime(df['Start Time']).dt.strftime('%m-%d-%YT%H:%M')
    df["End Time"] = pd.to_datetime(df['End Time']).dt.strftime('%m-%d-%YT%H:%M')

def drop_useless_cols(df):
    df = df[['Name', 'Section', 'Start Time', 'End Time', 'Building', 'Room', 'Capacity', 'Address', 'lat', 'long']]

def insert_address(df, address):
    df = pd.merge(df, address, on="Building")
    return df

def insert_beg_end(df):
    for date in set(df["Date"]):
        for classroom in set(df["Location"]):
            df.loc[len(df)] = [pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, date, '06:00', '6:00', classroom, df[df.Location == classroom].iloc[0]['Capacity'], pd.NA, pd.NA, pd.NA, pd.NA, df[df.Location == classroom].iloc[0]['Building'], df[df.Location == classroom].iloc[0]['Room'], df[df.Location == classroom].iloc[0]['Address'], df[df.Location == classroom].iloc[0]['Lat'], df[df.Location == classroom].iloc[0]['Long']]
            df.loc[len(df)] = [pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, date, '22:00', '22:00', classroom, df[df.Location == classroom].iloc[0]['Capacity'], pd.NA, pd.NA, pd.NA, pd.NA, df[df.Location == classroom].iloc[0]['Building'], df[df.Location == classroom].iloc[0]['Room'], df[df.Location == classroom].iloc[0]['Address'], df[df.Location == classroom].iloc[0]['Lat'], df[df.Location == classroom].iloc[0]['Long']]

def calculate_diff(df):
    pass

In [24]:
df = pd.read_csv("meetings.csv")

In [25]:
address = pd.read_csv("address.csv")

In [26]:
clean_file(df)

In [27]:
df = insert_address(df, address)

In [28]:
insert_beg_end(df)

In [33]:
df.tail()

,Name,Section,Type,Title,Note,Date,Published Start,Published End,Location,Capacity,Instructor / Organization,Email,Requested Services,Approved,Building,Room,Address,Lat,Long
10817,<NA>,<NA>,<NA>,<NA>,<NA>,03/30/2023,22:00,22:00,ABE B053,32,<NA>,<NA>,<NA>,<NA>,ABE,B053,225 S University Street,40.421846,40.421846
10818,<NA>,<NA>,<NA>,<NA>,<NA>,03/30/2023,06:00,6:00,KNOY B031,46,<NA>,<NA>,<NA>,<NA>,KNOY,B031,401 N Grant Street,40.427960,40.427960
10819,<NA>,<NA>,<NA>,<NA>,<NA>,03/30/2023,22:00,22:00,KNOY B031,46,<NA>,<NA>,<NA>,<NA>,KNOY,B031,401 N Grant Street,40.427960,40.427960
10820,<NA>,<NA>,<NA>,<NA>,<NA>,03/30/2023,06:00,6:00,FRNY B124,65,<NA>,<NA>,<NA>,<NA>,FRNY,B124,480 Stadium Mall Drive,40.429639,40.429639
10821,<NA>,<NA>,<NA>,<NA>,<NA>,03/30/2023,22:00,22:00,FRNY B124,65,<NA>,<NA>,<NA>,<NA>,FRNY,B124,480 Stadium Mall Drive,40.429639,40.429639


In [30]:
sorted_df = df.sort_values("Published Start")

In [31]:
date_list = []
for d_index, date in enumerate(sorted(list(set(df["Date"])))):
    class_list = [[] for i in range(len(sorted_df))]
    for c_index, classroom in enumerate(sorted(list(set(sorted_df["Location"])))):
        class_list[c_index] = sorted_df[(sorted_df["Location"] == classroom) & (sorted_df["Date"] == date)].reset_index()
    date_list.append(class_list)